In [104]:
import random
import numpy as np
import pandas as pd
import cvxpy as cp
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms


In [105]:
"""
Data Load
"""
cov_matrix = pd.read_excel("沪深300十年收盘价相关系数.xlsx", index_col=0)
cov_matrix

,000001.SZ,000002.SZ,000063.SZ,000069.SZ,000100.SZ,000157.SZ,000301.SZ,000338.SZ,000408.SZ,000425.SZ,...,601888.SH,601898.SH,601899.SH,601901.SH,601919.SH,601939.SH,601988.SH,601989.SH,601998.SH,603993.SH
000001.SZ,1.000,0.456,0.331,0.460,0.343,0.409,0.226,0.316,0.154,0.265,...,0.276,0.361,0.302,0.456,0.285,0.565,0.505,0.290,0.559,0.262
000002.SZ,0.456,1.000,0.286,0.593,0.263,0.378,0.183,0.309,0.110,0.247,...,0.223,0.295,0.257,0.348,0.241,0.444,0.377,0.244,0.395,0.215
000063.SZ,0.331,0.286,1.000,0.354,0.473,0.380,0.307,0.330,0.283,0.304,...,0.285,0.287,0.290,0.390,0.306,0.258,0.237,0.388,0.271,0.303
000069.SZ,0.460,0.593,0.354,1.000,0.360,0.478,0.325,0.385,0.250,0.344,...,0.271,0.444,0.364,0.444,0.341,0.416,0.386,0.406,0.405,0.304
000100.SZ,0.343,0.263,0.473,0.360,1.000,0.487,0.343,0.388,0.275,0.320,...,0.289,0.378,0.384,0.439,0.343,0.278,0.245,0.403,0.263,0.340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601939.SH,0.565,0.444,0.258,0.416,0.278,0.371,0.171,0.285,0.075,0.240,...,0.197,0.387,0.293,0.427,0.254,1.000,0.769,0.301,0.658,0.217
601988.SH,0.505,0.377,0.237,0.386,0.245,0.298,0.168,0.244,0.074,0.212,...,0.166,0.368,0.242,0.408,0.234,0.769,1.000,0.257,0.692,0.192
601989.SH,0.290,0.244,0.388,0.406,0.403,0.482,0.370,0.394,0.328,0.346,...,0.253,0.478,0.369,0.437,0.400,0.301,0.257,1.000,0.288,0.356
601998.SH,0.559,0.395,0.271,0.405,0.263,0.339,0.199,0.280,0.128,0.256,...,0.199,0.367,0.266,0.457,0.251,0.658,0.692,0.288,1.000,0.217


In [106]:
"""
Check Matrix Symmetry
"""
def check_symmetry(cov_matrix):
    # Check symmetry
    if not np.allclose(cov_matrix, cov_matrix.T):
        print("The covariance matrix is not symmetric.")
    else:
        print("The covariance matrix is symmetric.")
        return

    # Find non-symmetric pairs
    non_symmetric_pairs = []
    for i in range(len(cov_matrix)):
        for j in range(i+1, len(cov_matrix)):
            if not np.isclose(cov_matrix.iloc[i, j], cov_matrix.iloc[j, i]):
                row_asset = cov_matrix.index[i]
                col_asset = cov_matrix.columns[j]
                non_symmetric_pairs.append((row_asset, col_asset))
    for pair in non_symmetric_pairs:
        print("Non-symmetric pair:", pair)

# check_symmetry(cov_matrix)

In [107]:
"""
GA Initialization
"""
# Objective
creator.create("FitnessMulti", base.Fitness, weights=(-1.0,)) # MIN portfolio variance
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Individual Initialisation
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(cov_matrix.columns))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

C:\Tool\Python3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Tool\Python3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [108]:
"""
Test Model:
Min sum(wi*cov(ri,rj)), s.t. sum(wi)=1，0<=wi<=1
"""
# Portfolio Variance
def test_model(individual, cov_matrix):
    # extract sub matrix
    included_assets_indices = [index for index, include in enumerate(individual) if include == 1]
    sub_cov_matrix = cov_matrix.iloc[included_assets_indices, included_assets_indices]
    sub_cov_matrix = np.array(sub_cov_matrix)

    # bound and weight
    n = len(included_assets_indices)
    bounds_min = [0 for _ in range(n)]
    bounds_max = [1 for _ in range(n)]
    weights = cp.Variable(n)

    # define objective - minimize variance
    portfolio_variance = cp.quad_form(weights, sub_cov_matrix)
    objective = cp.Minimize(portfolio_variance)

    # constraints
    constraints = [cp.sum(weights) == 1, weights >= bounds_min, weights <= bounds_max]
    problem = cp.Problem(objective, constraints)

    # solve problem
    problem.solve()
    if problem.status == cp.OPTIMAL:
        optimal_weights = weights.value
        optimal_portfolio_variance = np.dot(optimal_weights.T, np.dot(sub_cov_matrix, optimal_weights))
    else:
        optimal_weights = None
        optimal_portfolio_variance = 9999

    return optimal_portfolio_variance, optimal_weights

In [109]:
"""
Fitness Function
"""
def evaluate(individual):
    portfolio_variance, weights = test_model(individual, cov_matrix)
    return portfolio_variance,

In [110]:
"""
Algorithm Setup
"""
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selNSGA2)

In [111]:
"""
Run GA
"""
def run_ga(pop_size, num_generations):
    pop = toolbox.population(n=pop_size)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    # check hof generation number (1)
    # generation_of_individual = {}
    # for gen in range(num_generations):
    #     # Check generation index
    #     print(f"Generation: {gen}")
    #     # Run one generation of the genetic algorithm
    #     algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=1, stats=stats, halloffame=hof, verbose=True)
    #     # Update the generation information for each individual in the hall of fame
    #     for ind in hof:
    #         if str(ind) not in generation_of_individual:
    #             generation_of_individual[str(ind)] = gen
    # return pop, hof, stats, generation_of_individual

    # without generation series check (2)
    pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=num_generations, stats=stats, halloffame=hof, verbose=True)

    return pop, hof, stats, logbook

population, pareto_front, stats, logbook = run_ga(500, 1000)

gen	nevals	avg         	std        	min         	max        
0  	500   	[0.19140966]	[0.0085306]	[0.17301721]	[0.2212003]
1  	391   	[0.19146608]	[0.0094499]	[0.1719073] 	[0.2212003]
2  	382   	[0.1914787] 	[0.0101349]	[0.17171419]	[0.22599549]
3  	364   	[0.19162869]	[0.01077476]	[0.17132749]	[0.22599549]
4  	360   	[0.19174574]	[0.01105392]	[0.17012715]	[0.22599549]
5  	389   	[0.19173016]	[0.01151043]	[0.17009276]	[0.22599549]
6  	396   	[0.19182902]	[0.01204043]	[0.17009276]	[0.2284591] 
7  	378   	[0.19174085]	[0.01240794]	[0.16973775]	[0.23965465]
8  	381   	[0.19188128]	[0.0125646] 	[0.16965076]	[0.23965465]
9  	372   	[0.19202977]	[0.01272833]	[0.17051346]	[0.24283142]
10 	382   	[0.19218279]	[0.01303313]	[0.17028396]	[0.24667524]
11 	395   	[0.19214643]	[0.01336278]	[0.16923671]	[0.24667524]
12 	380   	[0.19221862]	[0.013642]  	[0.16941579]	[0.24214892]
13 	387   	[0.19238463]	[0.0138102] 	[0.16941579]	[0.23992969]
14 	392   	[0.19240308]	[0.01404497]	[0.16956261]	[0.23992969]

KeyboardInterrupt: 

In [ ]:
"""
Pareto Front Output
"""
# Get weights info
def get_weights(individual):
    portfolio_variance, optimized_weights = test_model(individual, cov_matrix)
    full_weights = np.zeros(len(cov_matrix.columns))
    weight_index = 0
    for i, included in enumerate(individual):
        if included == 1:
            full_weights[i] = optimized_weights[weight_index]
            weight_index += 1
    return full_weights

# Retrieve and store weights
individual_weights = {str(ind): get_weights(ind) for ind in pareto_front}

# Print the individuals, corresponding weights and generation series number
for ind in pareto_front:
    print(f"Individual: {ind}")
    print(f"Weights: {individual_weights[str(ind)]}")
    print(f"Portfolio Variance: {test_model(ind, cov_matrix)[0]}")
    # print(f"Generation Index: {generation_of_individual[str(ind)]}")


In [ ]:
gen = logbook.select('gen')
min_fitness = logbook.select('min')

def normalize(data):
    min_val = np.min(data)
    max_val = np.max(data)
    return (data - min_val) / (max_val - min_val)

normalized_min_fitness = [normalize([fit[i] for fit in min_fitness]) for i in range(len(min_fitness[0]))]

plt.figure(figsize=(12, 6))

for obj_avg in normalized_min_fitness:
    plt.plot(gen, obj_avg, label=f'Normalized Portfolio Variance Average Fitness', marker='o')

plt.fill_between(gen, 0, 1, alpha=0.2, color='grey')

plt.title('Fitness Statistics over Generations')
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.legend()
plt.grid(True)
plt.show()